In [1]:
# Load reverse_query_index, reverse ranker
# Find subset of passages that are not in train set: take the first 100 of them
# For each passage, use reverse_query_index to find top 1000 nearest queries
# Load forward_passage_index, forward ranker
# Use forward_passage_index to find top k passages for each of the query, count += 1 if the passage is in top k
# Record the # of top k queries for each document 
# Plot histogram for distribution and compute the average

In [73]:
from annoy import AnnoyIndex
import torch
import sys
import random

sys.path.insert(0, '/home/jianx/search-exposure/forward_ranker/')
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from train import generate_sparse
from load_data import obj_reader, obj_writer
import network

from annoy import AnnoyIndex
from utils import print_message

EMBED_SIZE = 256
DEVICE = "cuda:1"
n_passage = 10000
n_query = 100
rank = 100

REVERSE_INDEX_PATH = "./results/128_query_index.ann"
REVERSE_RANKER_PATH = "./results/reverse200_500_500_0.001_256_10.model"
FORWARD_INDEX_PATH = "/home/jianx/data/annoy/128_passage_index.ann"
FORWARD_RANKER_PATH = "/home/jianx/data/results/100_1000_1000_0.001_256_10.model"
PASSAGE_DICT_PATH = "/home/jianx/data/passages.dict"
QUERY_TRAIN_DICT_PATH = "/home/jianx/data/queries_train.dict"
TRAIN_RANK_PATH = "/home/jianx/data/train_data/256_20000_100_100_training.csv"
REVERSE_MAP_PATH = "./results/128_qid_map.dict"
FORWARD_MAP_PATH = "/home/jianx/data/annoy/128_pid_map.dict"

In [8]:
def load_train(path):
    with open(path) as file:
        line = file.readline()
        my_dict = {}
        while line:
            tokens = line.split(",")
            pid = int(tokens[0])
            qid = int(tokens[1])
            rank = int(tokens[2].rstrip())
            if pid not in my_dict:
                my_dict[pid] = {}
            my_dict[pid][qid] = rank
            line = file.readline()
    return my_dict
def load():
    query_dict = obj_reader(QUERY_TRAIN_DICT_PATH)
    passage_dict = obj_reader(PASSAGE_DICT_PATH)
    train_rank_dict = load_train(TRAIN_RANK_PATH)
    return train_rank_dict, query_dict, passage_dict

In [3]:
import torch.nn as nn

NUM_HIDDEN_NODES = 64
NUM_HIDDEN_LAYERS = 3
DROPOUT_RATE = 0.1
FEAT_COUNT = 100000


# Define the network
class DSSM(torch.nn.Module):

    def __init__(self, embed_size):
        super(DSSM, self).__init__()

        layers = []
        last_dim = FEAT_COUNT
        for i in range(NUM_HIDDEN_LAYERS):
            layers.append(nn.Linear(last_dim, NUM_HIDDEN_NODES))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm(NUM_HIDDEN_NODES))
            layers.append(nn.Dropout(p=DROPOUT_RATE))
            last_dim = NUM_HIDDEN_NODES
        layers.append(nn.Linear(last_dim, embed_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

    def parameter_count(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [27]:
# Load reverse query index
reverse_query_index = AnnoyIndex(EMBED_SIZE, 'euclidean')
reverse_query_index.load(REVERSE_INDEX_PATH)
# Load reverse query index mapping dict
reverse_query_map = obj_reader(REVERSE_MAP_PATH)
# Load reverse ranker model
reverse_ranker = DSSM(embed_size=EMBED_SIZE)
reverse_ranker.load_state_dict(torch.load(REVERSE_RANKER_PATH))
reverse_ranker.to(DEVICE)
reverse_ranker.eval()
# Load forward passage index
forward_passage_index = AnnoyIndex(EMBED_SIZE, 'euclidean')
forward_passage_index.load(FORWARD_INDEX_PATH)
# Load forward passage index mapping dict 
forward_passage_map = obj_reader(FORWARD_MAP_PATH)
# Load forward ranker model
forward_ranker = network.DSSM(embed_size=EMBED_SIZE)
forward_ranker.load_state_dict(torch.load(FORWARD_RANKER_PATH))
forward_ranker.to(DEVICE)
forward_ranker.eval()
# Load train_rank, query, passage dict
train_rank_dict, query_dict, passage_dict = load()

In [19]:
# Find subset of passages that are not in train set: take the first 100 of them
train_passage_list = set(list(train_rank_dict.keys()))
all_passage_list = set(list(passage_dict.keys()))
test_passage_list = list(all_passage_list.difference(train_passage_list))

In [75]:
# For each passage, use reverse_query_index to find top 1000 nearest queries
# Use forward_passage_index to find top k passages for each of the query, count += 1 if the passage is in top k
# Record the # of top k queries for each document 
# Plot histogram for distribution and compute the average
random_test_passage = random.sample(test_passage_list, n_passage)
# random_test_passage = random.sample(list(train_passage_list), n_passage)
counter = 0
rankings = []
total_sum = 0
match_count = 0
non_zero = []
for i, pid in enumerate(random_test_passage):
    print_message("Processing passage No. " + str(i+1) + "/" + str(n_passage))
    embedding = reverse_ranker(generate_sparse(passage_dict[pid]).to(DEVICE)).detach()
    nearest_queries = reverse_query_index.get_nns_by_vector(embedding, n_query)
    matching_list = []

    for i, annoy_qid in enumerate(nearest_queries):
        qid = reverse_query_map[annoy_qid]
        top_list = forward_passage_index.get_nns_by_vector(forward_ranker(generate_sparse(query_dict[qid]).to(DEVICE)).detach(),
                                                   rank)
        is_matched = False
        for j, annoy_pid in enumerate(top_list):
            if forward_passage_map[annoy_pid] == pid:
                print("Match!!!!! Rank: " + str(j + 1))
                matching_list.append(j + 1)
                non_zero.append(j + 1)
                is_matched = True
                match_count += 1
                break
            if not is_matched:
                matching_list.append(0)
    total_sum += sum(matching_list)
    rankings.append(matching_list)
    counter += 1
print(match_count / n_passage)

[Jul 16, 00:37:21] Processing passage No. 1/10000
[Jul 16, 00:37:22] Processing passage No. 2/10000
[Jul 16, 00:37:23] Processing passage No. 3/10000
[Jul 16, 00:37:24] Processing passage No. 4/10000
[Jul 16, 00:37:25] Processing passage No. 5/10000
[Jul 16, 00:37:26] Processing passage No. 6/10000
[Jul 16, 00:37:27] Processing passage No. 7/10000
[Jul 16, 00:37:28] Processing passage No. 8/10000
[Jul 16, 00:37:29] Processing passage No. 9/10000
[Jul 16, 00:37:30] Processing passage No. 10/10000
[Jul 16, 00:37:31] Processing passage No. 11/10000
[Jul 16, 00:37:32] Processing passage No. 12/10000
[Jul 16, 00:37:33] Processing passage No. 13/10000
[Jul 16, 00:37:34] Processing passage No. 14/10000
[Jul 16, 00:37:35] Processing passage No. 15/10000
[Jul 16, 00:37:36] Processing passage No. 16/10000
[Jul 16, 00:37:37] Processing passage No. 17/10000
[Jul 16, 00:37:38] Processing passage No. 18/10000
[Jul 16, 00:37:39] Processing passage No. 19/10000
[Jul 16, 00:37:40] Processing passage No

[Jul 16, 00:40:08] Processing passage No. 161/10000
[Jul 16, 00:40:09] Processing passage No. 162/10000
[Jul 16, 00:40:11] Processing passage No. 163/10000
[Jul 16, 00:40:12] Processing passage No. 164/10000
[Jul 16, 00:40:13] Processing passage No. 165/10000
[Jul 16, 00:40:14] Processing passage No. 166/10000
[Jul 16, 00:40:15] Processing passage No. 167/10000
[Jul 16, 00:40:16] Processing passage No. 168/10000
[Jul 16, 00:40:17] Processing passage No. 169/10000
[Jul 16, 00:40:18] Processing passage No. 170/10000
[Jul 16, 00:40:19] Processing passage No. 171/10000
[Jul 16, 00:40:20] Processing passage No. 172/10000
[Jul 16, 00:40:21] Processing passage No. 173/10000
[Jul 16, 00:40:22] Processing passage No. 174/10000
[Jul 16, 00:40:23] Processing passage No. 175/10000
[Jul 16, 00:40:24] Processing passage No. 176/10000
[Jul 16, 00:40:25] Processing passage No. 177/10000
[Jul 16, 00:40:26] Processing passage No. 178/10000
[Jul 16, 00:40:27] Processing passage No. 179/10000
[Jul 16, 00:

[Jul 16, 00:42:55] Processing passage No. 319/10000
[Jul 16, 00:42:56] Processing passage No. 320/10000
[Jul 16, 00:42:57] Processing passage No. 321/10000
[Jul 16, 00:42:58] Processing passage No. 322/10000
[Jul 16, 00:42:59] Processing passage No. 323/10000
[Jul 16, 00:43:00] Processing passage No. 324/10000
[Jul 16, 00:43:01] Processing passage No. 325/10000
[Jul 16, 00:43:02] Processing passage No. 326/10000
[Jul 16, 00:43:03] Processing passage No. 327/10000
[Jul 16, 00:43:04] Processing passage No. 328/10000
[Jul 16, 00:43:05] Processing passage No. 329/10000
[Jul 16, 00:43:06] Processing passage No. 330/10000
[Jul 16, 00:43:07] Processing passage No. 331/10000
[Jul 16, 00:43:08] Processing passage No. 332/10000
[Jul 16, 00:43:09] Processing passage No. 333/10000
[Jul 16, 00:43:10] Processing passage No. 334/10000
[Jul 16, 00:43:12] Processing passage No. 335/10000
[Jul 16, 00:43:13] Processing passage No. 336/10000
[Jul 16, 00:43:14] Processing passage No. 337/10000
[Jul 16, 00:

[Jul 16, 00:45:41] Processing passage No. 476/10000
[Jul 16, 00:45:42] Processing passage No. 477/10000
[Jul 16, 00:45:43] Processing passage No. 478/10000
[Jul 16, 00:45:44] Processing passage No. 479/10000
[Jul 16, 00:45:45] Processing passage No. 480/10000
[Jul 16, 00:45:46] Processing passage No. 481/10000
[Jul 16, 00:45:47] Processing passage No. 482/10000
[Jul 16, 00:45:48] Processing passage No. 483/10000
[Jul 16, 00:45:49] Processing passage No. 484/10000
[Jul 16, 00:45:50] Processing passage No. 485/10000
[Jul 16, 00:45:51] Processing passage No. 486/10000
[Jul 16, 00:45:52] Processing passage No. 487/10000
[Jul 16, 00:45:53] Processing passage No. 488/10000
[Jul 16, 00:45:54] Processing passage No. 489/10000
[Jul 16, 00:45:55] Processing passage No. 490/10000
[Jul 16, 00:45:57] Processing passage No. 491/10000
[Jul 16, 00:45:58] Processing passage No. 492/10000
[Jul 16, 00:45:59] Processing passage No. 493/10000
[Jul 16, 00:46:00] Processing passage No. 494/10000
[Jul 16, 00:

[Jul 16, 00:48:27] Processing passage No. 634/10000
[Jul 16, 00:48:28] Processing passage No. 635/10000
[Jul 16, 00:48:29] Processing passage No. 636/10000
[Jul 16, 00:48:30] Processing passage No. 637/10000
[Jul 16, 00:48:31] Processing passage No. 638/10000
[Jul 16, 00:48:32] Processing passage No. 639/10000
[Jul 16, 00:48:33] Processing passage No. 640/10000
[Jul 16, 00:48:34] Processing passage No. 641/10000
[Jul 16, 00:48:35] Processing passage No. 642/10000
[Jul 16, 00:48:36] Processing passage No. 643/10000
[Jul 16, 00:48:37] Processing passage No. 644/10000
[Jul 16, 00:48:38] Processing passage No. 645/10000
[Jul 16, 00:48:39] Processing passage No. 646/10000
[Jul 16, 00:48:40] Processing passage No. 647/10000
[Jul 16, 00:48:42] Processing passage No. 648/10000
[Jul 16, 00:48:43] Processing passage No. 649/10000
[Jul 16, 00:48:44] Processing passage No. 650/10000
[Jul 16, 00:48:45] Processing passage No. 651/10000
[Jul 16, 00:48:46] Processing passage No. 652/10000
[Jul 16, 00:

...

[Jul 16, 01:51:33] Processing passage No. 4206/10000
[Jul 16, 01:51:34] Processing passage No. 4207/10000
[Jul 16, 01:51:35] Processing passage No. 4208/10000
[Jul 16, 01:51:36] Processing passage No. 4209/10000
[Jul 16, 01:51:37] Processing passage No. 4210/10000
[Jul 16, 01:51:38] Processing passage No. 4211/10000
[Jul 16, 01:51:39] Processing passage No. 4212/10000
[Jul 16, 01:51:40] Processing passage No. 4213/10000
[Jul 16, 01:51:41] Processing passage No. 4214/10000
[Jul 16, 01:51:43] Processing passage No. 4215/10000
[Jul 16, 01:51:44] Processing passage No. 4216/10000
[Jul 16, 01:51:45] Processing passage No. 4217/10000
[Jul 16, 01:51:46] Processing passage No. 4218/10000
[Jul 16, 01:51:47] Processing passage No. 4219/10000
[Jul 16, 01:51:48] Processing passage No. 4220/10000
[Jul 16, 01:51:49] Processing passage No. 4221/10000
[Jul 16, 01:51:50] Processing passage No. 4222/10000
[Jul 16, 01:51:51] Processing passage No. 4223/10000
[Jul 16, 01:51:52] Processing passage No. 4224

[Jul 16, 01:54:17] Processing passage No. 4361/10000
[Jul 16, 01:54:18] Processing passage No. 4362/10000
[Jul 16, 01:54:19] Processing passage No. 4363/10000
[Jul 16, 01:54:20] Processing passage No. 4364/10000
[Jul 16, 01:54:21] Processing passage No. 4365/10000
[Jul 16, 01:54:22] Processing passage No. 4366/10000
[Jul 16, 01:54:23] Processing passage No. 4367/10000
[Jul 16, 01:54:24] Processing passage No. 4368/10000
[Jul 16, 01:54:25] Processing passage No. 4369/10000
[Jul 16, 01:54:26] Processing passage No. 4370/10000
[Jul 16, 01:54:28] Processing passage No. 4371/10000
[Jul 16, 01:54:29] Processing passage No. 4372/10000
[Jul 16, 01:54:30] Processing passage No. 4373/10000
[Jul 16, 01:54:31] Processing passage No. 4374/10000
[Jul 16, 01:54:32] Processing passage No. 4375/10000
[Jul 16, 01:54:33] Processing passage No. 4376/10000
[Jul 16, 01:54:34] Processing passage No. 4377/10000
[Jul 16, 01:54:35] Processing passage No. 4378/10000
[Jul 16, 01:54:36] Processing passage No. 4379

[Jul 16, 01:57:00] Processing passage No. 4516/10000
[Jul 16, 01:57:01] Processing passage No. 4517/10000
[Jul 16, 01:57:03] Processing passage No. 4518/10000
[Jul 16, 01:57:04] Processing passage No. 4519/10000
[Jul 16, 01:57:05] Processing passage No. 4520/10000
[Jul 16, 01:57:06] Processing passage No. 4521/10000
[Jul 16, 01:57:07] Processing passage No. 4522/10000
[Jul 16, 01:57:08] Processing passage No. 4523/10000
[Jul 16, 01:57:09] Processing passage No. 4524/10000
[Jul 16, 01:57:10] Processing passage No. 4525/10000
[Jul 16, 01:57:11] Processing passage No. 4526/10000
[Jul 16, 01:57:12] Processing passage No. 4527/10000
[Jul 16, 01:57:13] Processing passage No. 4528/10000
[Jul 16, 01:57:14] Processing passage No. 4529/10000
[Jul 16, 01:57:15] Processing passage No. 4530/10000
[Jul 16, 01:57:16] Processing passage No. 4531/10000
[Jul 16, 01:57:17] Processing passage No. 4532/10000
[Jul 16, 01:57:18] Processing passage No. 4533/10000
[Jul 16, 01:57:19] Processing passage No. 4534

[Jul 16, 01:59:44] Processing passage No. 4671/10000
[Jul 16, 01:59:45] Processing passage No. 4672/10000


KeyboardInterrupt: 

In [76]:
non_zero_rankings = []
for i in range(len(rankings)):
    for j in rankings[i]:
        if j != 0:
            non_zero_rankings.append(j)
print(non_zero_rankings)

[30, 60, 1, 36, 39, 15, 20]


In [77]:
len(non_zero_rankings) / len(rankings)

0.00149860843502462